In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

In [ ]:
import os
import numpy as np
import pandas as pd
import json
import warnings
import logging
import gc
import random
import math
import re
import ast
from tqdm import tqdm
from typing import Optional
from datetime import datetime
import pickle


from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import jaccard_score, f1_score, accuracy_score, recall_score, precision_score, confusion_matrix
from sklearn.model_selection import train_test_split

In [ ]:
!pip install transformers

In [ ]:
import nltk
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.meteor_score import meteor_score
# from rouge_score.rouge_scorer import RougeScorer

warnings.filterwarnings("ignore")

import torch
from torch.utils.data import DataLoader, TensorDataset

from transformers import (
    BartTokenizerFast,
    AdamW
)

if torch.cuda.is_available():
    DEVICE = torch.device("cuda")
    print("Using GPU")

else:
    DEVICE = torch.device("cpu")
    print("Using CPU")

foldNum = 0



SOURCE_MAX_LEN = 500
# TARGET_MAX_LEN = 50
# MAX_UTTERANCES = 25

ACOUSTIC_DIM = 768
ACOUSTIC_MAX_LEN = 1000



VISUAL_DIM = 2048
VISUAL_MAX_LEN = 480



import random




LEARNING_RATE = 1e-4
# LEARNING_RATE = 1e-5
# LEARNING_RATE = 1e-3


# VALID_LEN = 69

# BASE_LEARNING_RATE = 5e-6
# NEW_LEARNING_RATE = 5e-5
# WEIGHT_DECAY = 1e-4

# NUM_BEAMS = 5
# EARLY_STOPPING = True
# NO_REPEAT_NGRAM_SIZE = 3

# EARLY_STOPPING_THRESHOLD = 5

In [ ]:
def set_random_seed(seed: int):
    print("Seed : {}".format(seed))

    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.enabled = False
    torch.backends.cudnn.deterministic = True

    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

# set_random_seed(42)
# set_random_seed(123)
# set_random_seed(12345)
set_random_seed(994)

import torch.nn as nn
import torch.nn.functional as F
import torch.utils.checkpoint
from torch.nn import CrossEntropyLoss, MSELoss

from typing import Any, Callable, Dict, Iterable, List, Optional, Tuple, Union

from transformers.modeling_utils import PreTrainedModel, unwrap_model

# from transformers.models.roberta.configuration_roberta import RobertaConfig
# # from transformers.models.roberta.tokenization_roberta import RobertaTokenizer
# from transformers.models.roberta.modeling_roberta import RobertaLayer, RobertaEmbeddings, RobertaPooler, RobertaPreTrainedModel
# # from transformers.models.roberta.modeling_roberta import RobertaEmbeddings, RobertaPreTrainedModel
# from transformers import RobertaTokenizer, RobertaModel
# from transformers.modeling_outputs import (
#     BaseModelOutputWithPastAndCrossAttentions,
#     BaseModelOutputWithPoolingAndCrossAttentions
# )

from transformers.models.bert.configuration_bert import BertConfig

from transformers.models.bert.modeling_bert import BertLayer, BertEmbeddings, BertPooler, BertPreTrainedModel

from transformers import BertTokenizer, BertModel

from transformers.modeling_outputs import (
    BaseModelOutputWithPastAndCrossAttentions,
    BaseModelOutputWithPoolingAndCrossAttentions
)

from transformers import AutoModelForSequenceClassification
from transformers.pytorch_utils import apply_chunking_to_forward, find_pruneable_heads_and_indices, prune_linear_layer
from transformers.activations import ACT2FN

In [ ]:
# from transformer_encoder import TransformerEncoder

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

In [ ]:
# bert_tokenizer  = BertTokenizer.from_pretrained("bert-base-cased")
# bert_model = BertModel.from_pretrained("bert-base-cased")
# # bert_model

In [ ]:
# class ContextAwareAttention(nn.Module):

#     def __init__(self,
#                  dim_model : int,
#                  dim_context : int,
#                  dropout_rate : Optional[float] = 0.0 ):

#         super(ContextAwareAttention, self).__init__()

#         self.dim_model = dim_model
#         self.dim_context = dim_context
#         self.dropout_rate = dropout_rate
#         self.attention_layer = nn.MultiheadAttention(embed_dim=self.dim_model,
#                                                      num_heads = 1,
#                                                      dropout = self.dropout_rate,
#                                                      bias = True,
#                                                     add_zero_attn=False,
#                                                     batch_first=True,
#                                                     device=DEVICE
#         )

#         self.u_k = nn.Linear(self.dim_context, self.dim_model, bias = False)
#         self.w1_k = nn.Linear(self.dim_model, 1, bias=False)
#         self.w2_k = nn.Linear(self.dim_model, 1, bias=False)

#         self.u_v = nn.Linear(self.dim_context, self.dim_model, bias=False)
#         self.w1_v = nn.Linear(self.dim_model, 1, bias = False)
#         self.w2_v = nn.Linear(self.dim_model, 1, bias = False)

#     def forward(self, q, k, v, context):

#         # print("Context shape : ", context.shape)
#         # print("Dim context : ", self.dim_context, " : Dim model : ", self.dim_model)
#         key_context = self.u_k(context)
#         # print("Context shape below key context : ", key_context.shape)
#         value_context = self.u_v(context)

#         lambda_k = F.sigmoid(self.w1_k(k) + self.w2_k(key_context))
#         lambda_v = F.sigmoid(self.w1_v(v) + self.w2_v(value_context))

#         k_cap = (1-lambda_k) * k + (lambda_k) * key_context
#         v_cap = (1-lambda_v) * v + (lambda_v) * value_context

#         attention_output, _ = self.attention_layer(query = q,
#                                                    key = k_cap,
#                                                    value = v_cap)

#         return attention_output



In [ ]:
# class MAF_acoustic(nn.Module):
#     def __init__(self,
#                 dim_model,
#                 dropout_rate):
#         super(MAF_acoustic, self).__init__()
#         self.dropout_rate = dropout_rate

#         self.acoustic_context_transform = nn.Linear(ACOUSTIC_MAX_LEN, SOURCE_MAX_LEN, bias = False)
#         # self.visual_context_transform = nn.Linear(VISUAL_MAX_LEN, SOURCE_MAX_LEN, bias = False)

#         self.acoustic_context_attention = ContextAwareAttention(dim_model=dim_model,
#                                                                 dim_context=ACOUSTIC_DIM,
#                                                                 dropout_rate=dropout_rate)

#         # self.visual_context_attention = ContextAwareAttention(dim_model=dim_model,
#         #                                                     dim_context=VISUAL_DIM,
#         #                                                     dropout_rate=dropout_rate)

#         self.acoustic_gate = nn.Linear(2*dim_model, dim_model)
#         # self.visual_gate = nn.Linear(2*dim_model, dim_model)
#         self.dropout_layer = nn.Dropout(dropout_rate)
#         self.final_layer_norm = nn.LayerNorm(dim_model)

#     def forward(self,
#                 text_input,
#                 acoustic_context):

#         # print("Acoustic context shape (A) : ", acoustic_context.shape)

#         acoustic_context = acoustic_context.permute(0,2,1)
#         acoustic_context = self.acoustic_context_transform(acoustic_context.float())
#         acoustic_context = acoustic_context.permute(0,2,1)

#         audio_out = self.acoustic_context_attention(q=text_input,
#                                                     k=text_input,
#                                                     v=text_input,
#                                                     context=acoustic_context)
#         # print("Audio out (A) : ", audio_out.shape)

#         # print("Visual context shape : ", visual_context.shape)
#         # visual_context = visual_context.permute(0,2,1)
#         # visual_context = self.visual_context_transform(visual_context.float())
#         # visual_context = visual_context.permute(0,2,1)

#         # video_out = self.visual_context_attention(q=text_input,
#         #                                             k=text_input,
#         #                                             v=text_input,
#         #                                             context=visual_context)

#         # print("Video out shape : ", video_out.shape)
#         # print("Text input shape : ", text_input.shape)
#         weight_a = F.sigmoid(self.acoustic_gate(torch.cat([text_input, audio_out], dim=-1)))
#         # weight_v = F.sigmoid(self.visual_gate(torch.cat([text_input, video_out], dim=-1)))

#         # output = self.final_layer_norm(text_input + weight_a * audio_out + weight_v * video_out)

#         output = self.final_layer_norm(text_input + weight_a * audio_out)

#         return output

In [ ]:
# class MAF_visual(nn.Module):
#     def __init__(self,
#                 dim_model,
#                 dropout_rate):
#         super(MAF_visual, self).__init__()
#         self.dropout_rate = dropout_rate

#         # self.acoustic_context_transform = nn.Linear(ACOUSTIC_MAX_LEN, SOURCE_MAX_LEN, bias = False)
#         self.visual_context_transform = nn.Linear(VISUAL_MAX_LEN, SOURCE_MAX_LEN, bias = False)

#         # self.acoustic_context_attention = ContextAwareAttention(dim_model=dim_model,
#         #                                                         dim_context=ACOUSTIC_DIM,
#         #                                                         dropout_rate=dropout_rate)

#         self.visual_context_attention = ContextAwareAttention(dim_model=dim_model,
#                                                             dim_context=VISUAL_DIM,
#                                                             dropout_rate=dropout_rate)

#         # self.acoustic_gate = nn.Linear(2*dim_model, dim_model)
#         self.visual_gate = nn.Linear(2*dim_model, dim_model)
#         self.dropout_layer = nn.Dropout(dropout_rate)
#         self.final_layer_norm = nn.LayerNorm(dim_model)

#     def forward(self,
#                 text_input,
#                 visual_context):

#         # print("Acoustic context shape (A) : ", acoustic_context.shape)

#         # acoustic_context = acoustic_context.permute(0,2,1)
#         # acoustic_context = self.acoustic_context_transform(acoustic_context.float())
#         # acoustic_context = acoustic_context.permute(0,2,1)

#         # audio_out = self.acoustic_context_attention(q=text_input,
#         #                                             k=text_input,
#         #                                             v=text_input,
#         #                                             context=acoustic_context)
#         # print("Audio out (A) : ", audio_out.shape)

#         # print("Visual context shape : ", visual_context.shape)
#         visual_context = visual_context.permute(0,2,1)
#         visual_context = self.visual_context_transform(visual_context.float())
#         visual_context = visual_context.permute(0,2,1)

#         video_out = self.visual_context_attention(q=text_input,
#                                                     k=text_input,
#                                                     v=text_input,
#                                                     context=visual_context)

#         # print("Video out shape : ", video_out.shape)
#         # print("Text input shape : ", text_input.shape)
#         # weight_a = F.sigmoid(self.acoustic_gate(torch.cat([text_input, audio_out], dim=-1)))
#         weight_v = F.sigmoid(self.visual_gate(torch.cat([text_input, video_out], dim=-1)))

#         # output = self.final_layer_norm(text_input + weight_a * audio_out + weight_v * video_out)

#         # q3 = weight_v * video_out
#         # print('weight_v shape : ', weight_v.shape)
#         # print('weight_v * video_out shape : ', q3.shape)

#         output = self.final_layer_norm(text_input  + weight_v * video_out)

#         return output

In [ ]:
class MultimodalBertEncoder(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config
        self.layer = nn.ModuleList([BertLayer(config) for _ in range(config.num_hidden_layers)])
        self.gradient_checkpointing = False

        self.fusion_at_layer9 = [8]
        # self.fusion_at_layer10 = [11]

        # self.fusion_at_layer9 = [19]
        # self.fusion_at_layer10 = [20]


        self.acoustic_context_transform = nn.Linear(ACOUSTIC_MAX_LEN, SOURCE_MAX_LEN, bias = False)
        self.visual_context_transform = nn.Linear(VISUAL_MAX_LEN, SOURCE_MAX_LEN, bias = False)

        self.acoustic_dim = nn.Linear(ACOUSTIC_DIM, 768, bias = False)
        self.visual_dim = nn.Linear(VISUAL_DIM, 768, bias = False)

        self.concat_linear = nn.Linear(3*768, 768, bias = False)

        # self.MAF_layer9 = MAF_acoustic(dim_model=self.config.hidden_size,
        #                      dropout_rate=0.2)

        # self.MAF_layer10 = MAF_visual(dim_model=self.config.hidden_size,
        #                      dropout_rate=0.2)

    def forward(self,
               hidden_states : torch.Tensor,
               attention_mask: Optional[torch.FloatTensor] = None,
               acoustic_input: Optional[torch.FloatTensor] = None,
               visual_input: Optional[torch.FloatTensor] = None,
               head_mask: Optional[torch.FloatTensor] = None,
               encoder_hidden_states: Optional[torch.FloatTensor] = None,
               encoder_attention_mask: Optional[torch.FloatTensor] = None,
               past_key_values: Optional[Tuple[Tuple[torch.FloatTensor]]] = None,
               use_cache: Optional[bool] = False,
               output_attentions: Optional[bool] = False,
               output_hidden_states: Optional[bool] = False,
               return_dict: Optional[bool] = True
               ) -> Union[Tuple[torch.Tensor], BaseModelOutputWithPastAndCrossAttentions]:

        all_hidden_states = () if output_hidden_states else None
        all_self_attentions = () if output_attentions else None
        all_cross_attentions = () if output_attentions and self.config.add_cross_attention else None

        next_decoder_cache = () if use_cache else None
        for i, layer_module in enumerate(self.layer):
            # print("i : ", i)
            if i in self.fusion_at_layer9:
                    # print("Inside layer 9")
                    # print("Acoustic input shape (B) : ", acoustic_input)
                    # acoustic_input = self.acoustic_transformer(acoustic_input)[-1]
                    # print("Acoustic input shape (C) : ", acoustic_input)

                    # visual_input = self.visual_transformer(visual_input)[-1]
                    # print("====Idx inside fusion at layer :", idx)

                      acoustic_input = acoustic_input.permute(0,2,1)
                      acoustic_input = self.acoustic_context_transform(acoustic_input.float())
                      acoustic_input = acoustic_input.permute(0,2,1)

                      acoustic_input = self.acoustic_dim(acoustic_input)

                      # print("acoustic_input shape : ", acoustic_input.shape)

                      visual_input = visual_input.permute(0,2,1)
                      visual_input = self.visual_context_transform(visual_input.float())
                      visual_input = visual_input.permute(0,2,1)

                      visual_input = self.visual_dim(visual_input)

                      # print("visual input shape : ", visual_input.shape)
                      concat = torch.concat([hidden_states, acoustic_input, visual_input], dim = -1)

                      # print("concat shape : ", concat.shape)

                      hidden_states = self.concat_linear(concat)

                      # print('hidden states shape : ', hidden_states.shape)








            # if i in self.fusion_at_layer10:
            #         # print("Acoustic input shape (B) : ", acoustic_input)
            #         # acoustic_input = self.acoustic_transformer(acoustic_input)[-1]
            #         # print("Acoustic input shape (C) : ", acoustic_input)

            #         # visual_input = self.visual_transformer(visual_input)[-1]
            #         # print("====Idx inside fusion at layer :", idx)
            #         hidden_states = self.MAF_layer10(text_input = hidden_states,
            #                                        visual_context = visual_input)


            if output_hidden_states:
                all_hidden_states = all_hidden_states + (hidden_states,)

            layer_head_mask = head_mask[i] if head_mask is not None else None
            past_key_value = past_key_values[i] if past_key_values is not None else None

            if self.gradient_checkpointing and self.training:

                if use_cache:
                    # logger.warning(
                    #     "`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`..."
                    # )
                    use_cache = False

                def create_custom_forward(module):
                    def custom_forward(*inputs):
                        return module(*inputs, past_key_value, output_attentions)

                    return custom_forward

                layer_outputs = torch.utils.checkpoint.checkpoint(
                    create_custom_forward(layer_module),
                    hidden_states,
                    attention_mask,
                    layer_head_mask,
                    encoder_hidden_states,
                    encoder_attention_mask
                )

            else:
                # print("hidden states shape : ", hidden_states.shape)
                # print('attention_mask shape : ', attention_mask.shape)
                layer_outputs = layer_module(
                    hidden_states,
                    attention_mask,
                    layer_head_mask,
                    encoder_hidden_states,
                    encoder_attention_mask,
                    past_key_value,
                    output_attentions,


                )

            hidden_states = layer_outputs[0]

            if use_cache:
                next_decoder_cache += (layer_outputs[-1],)
            if output_attentions:
                all_self_attentions = all_self_attentions + (layer_outputs[1],)
                if self.config.add_cross_attention:
                    all_cross_attentions = all_cross_attentions + (layer_outputs[2],)

        if output_hidden_states:
            all_hidden_states = all_hidden_states + (hidden_states,)

        if not return_dict:
            return tuple(
                v
                for v in [
                    hidden_states,
                    next_decoder_cache,
                    all_hidden_states,
                    all_self_attentions,
                    all_cross_attentions,

                ]
                if v is not None
            )

        return BaseModelOutputWithPastAndCrossAttentions(
            last_hidden_state = hidden_states,
            past_key_values = next_decoder_cache,
            hidden_states = all_hidden_states,
            attentions = all_self_attentions,
            cross_attentions = all_cross_attentions

        )

In [ ]:
class MultiModalBertClassification(nn.Module):
    def __init__(self, input_dim, num_classes):
        super().__init__()
        self.dense = nn.Linear(input_dim, num_classes)
    def forward(self, hidden_states):
        hidden_states = torch.relu(hidden_states)
        hidden_states = self.dense(hidden_states)
        return hidden_states


In [ ]:
class MultiModalBertModel(BertPreTrainedModel):
    def __init__(self, config):

        super().__init__(config)

        self.config = config

        self.embeddings = BertEmbeddings(config)
        self.encoder = MultimodalBertEncoder(config)
        # self.encoder = BertEncoder(config)

        self.output = MultiModalBertClassification(config.hidden_size, num_classes = 2)

        self.post_init()

    def get_input_embeddings(self):
        return self.embeddings.word_embeddings

    def set_input_embeddings(self, value):
        self.embeddings.word_embeddings = value

    def _prune_heads(self, heads_to_prune):

        for layer, heads in heads_to_prune.items():
            self.encoder.layer[layer].attention.prune_heads(heads)

    def forward(
        self,
        input_ids: Optional[torch.Tensor] = None,
        attention_mask: Optional[torch.Tensor] = None,
        acoustic_input: Optional[torch.Tensor] = None,
        visual_input: Optional[torch.Tensor] = None,
        labels: Optional[torch.Tensor] = None,
        token_type_ids: Optional[torch.Tensor] = None,
        position_ids: Optional[torch.Tensor] = None,
        head_mask: Optional[torch.Tensor] = None,
        inputs_embeds: Optional[torch.Tensor] = None,
        encoder_hidden_states: Optional[torch.Tensor] = None,
        encoder_attention_mask: Optional[torch.Tensor] = None,
        past_key_values: Optional[List[torch.FloatTensor]] = None,
        use_cache: Optional[bool] = None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None
    )   -> Union[Tuple[torch.Tensor], BaseModelOutputWithPoolingAndCrossAttentions]:

        output_attentions = output_attentions if output_attentions is not None else self.config.output_attentions
        output_hidden_states = (
            output_hidden_states if output_hidden_states is not None else self.config.output_hidden_states
        )
        return_dict = return_dict if return_dict is not None else self.config.output_hidden_states

        if self.config.is_decoder:
            use_cache = use_cache if use_cache is not None else self.config.use_cache
        else:
            use_cache = False

        if input_ids is not None and inputs_embeds is not None:
            raise ValueError('You can not  specify both input_ids and input_embeds at the same time')
        elif input_ids is not None:
            input_shape = input_ids.size()
        elif inputs_embeds is not None:
            input_shape = inputs_embeds.size()[:-1]
        else:
            raise ValueError('You have to specify either input_ids or inputs_embeds')

        batch_size, seq_length = input_shape
        device = input_ids.device if input_ids is not None else inputs_embeds.device

        past_key_value_length = past_key_values[0][0].shape[2] if past_key_values is not None else 0

        if attention_mask is None:
            attention_mask = torch.ones(((batch_size, seq_length + past_key_value_length)), device = device)

        if token_type_ids is None:
            if hasattr(self.embeddings, 'token_type_ids'):
                buffered_token_type_ids = self.embeddings.token_type_ids[:, :seq_length]
                buffered_token_type_ids_expanded = buffered_token_type_ids.expand(batch_size, seq_length)
                token_type_ids = buffered_token_type_ids_expanded
            else:
                token_type_ids = torch.zeros(input_shape, dtype = torch.long, device = device)

        extended_attention_mask: torch.Tensor = self.get_extended_attention_mask(attention_mask, input_shape)

        # print("attention mask shape : ", attention_mask.shape)
        # print("extended attention mask shape : ", extended_attention_mask.shape)

        if self.config.is_decoder and encoder_hidden_states is not None:
            encoder_batch_size, encoder_sequence_length, _ = encoder_hidden_states.size()
            encoder_hidden_shape = (encoder_batch_size, encoder_sequence_length)
            if encoder_attention_mask is None:
                encoder_attention_mask = torch.ones(encoder_hidden_shape, device = device)

            encoder_extended_attention_mask = self.invert_attention_mask(encoder_attention_mask)

        else:
            encoder_extended_attention_mask = None

        head_mask = self.get_head_mask(head_mask, self.config.num_hidden_layers)

        embedding_output = self.embeddings(
            input_ids = input_ids,
            position_ids = position_ids,
            token_type_ids = token_type_ids,
            inputs_embeds = inputs_embeds,
            past_key_values_length = past_key_value_length
        )

        # print("attention mask shape 2 : ", attention_mask.shape)
        encoder_outputs = self.encoder(
            embedding_output,
            attention_mask = extended_attention_mask,
            acoustic_input = acoustic_input,
            visual_input = visual_input,
            head_mask = head_mask,
            encoder_hidden_states = encoder_hidden_states,
            encoder_attention_mask = encoder_extended_attention_mask,
            past_key_values = past_key_values,
            use_cache = use_cache,
            output_attentions = output_attentions,
            output_hidden_states = output_hidden_states,
            return_dict = return_dict
        )

        sequence_output = encoder_outputs[0]
        pooled_output = self.output(sequence_output)

        loss_fn = torch.nn.CrossEntropyLoss()

        # print("pooled output shape : ", pooled_output.shape)
        # print("labels shape : ", labels.shape)
        pooled_output = pooled_output[:, 0, :]
        loss = loss_fn(pooled_output, labels)

        temp_dict = {}

        temp_dict['logits'] = pooled_output
        temp_dict['loss'] = loss

        return temp_dict


In [ ]:
# def audio_video_broadcast(x):
# #     z = torch.empty()
#     temp_all = torch.Tensor()
#     for j in range(x.shape[0]):
#         print("j : ", j)
#         temp_x = x[j,:]
# #         print("Temp x shape : ", temp_x.shape)
#         temp_x = torch.tensor(temp_x, dtype=torch.float)
#         temp_x = torch.broadcast_to(temp_x, (SOURCE_MAX_LEN, temp_x.shape[0]))
# #         print("Temp x shape : ", temp_x.shape)
#         temp_x = temp_x.unsqueeze(0)
# #         print("Temp x shape : ", temp_x.shape)

#         if(j==0):
#             temp_all = temp_x
#         else:
#             temp_all = torch.cat([temp_all, temp_x], dim = 0)


#     return temp_all

In [ ]:
foldNum

In [ ]:
with open('/content/drive/MyDrive/Colab Notebooks/32/train_audio_fold_'+str(foldNum)+'.p', 'rb') as f:
  train_audio_data_utterance1 = pickle.load(f)


In [ ]:
type(train_audio_data_utterance1)

In [ ]:
len(train_audio_data_utterance1)

In [ ]:
with open('/content/drive/MyDrive/Colab Notebooks/32/test_audio_fold_'+str(foldNum)+'.p', 'rb') as f:
  test_audio_data_utterance1 = pickle.load(f)

In [ ]:
len(test_audio_data_utterance1)

In [ ]:
len(test_audio_data_utterance1)

In [ ]:
with open('/content/drive/MyDrive/Colab Notebooks/32/train_video_fold_'+str(foldNum)+'.p', 'rb') as f:
  train_image_data_utterance1 = pickle.load(f)

In [ ]:
len(train_image_data_utterance1)

In [ ]:
with open('/content/drive/MyDrive/Colab Notebooks/32/test_video_fold_'+str(foldNum)+'.p', 'rb') as f:
  test_image_data_utterance1 = pickle.load(f)

In [ ]:
len(test_image_data_utterance1)

In [ ]:
tp = torch.ones(4,5)
tp

In [ ]:
torch.zeros(5 - tp.shape[0], 5)

In [ ]:
torch.cat([tp, torch.zeros(5 - tp.shape[0], 5)])

In [ ]:
def pad_seq(tensor, dim, max_len):
  if max_len > tensor.shape[0] :
    return torch.cat([tensor, torch.zeros(max_len - tensor.shape[0], dim)])
  else:
    return tensor[:max_len]

In [ ]:
ACOUSTIC_DIM

In [ ]:
train_audio_data_utterance1 = torch.stack([pad_seq(torch.tensor(a, dtype = torch.float),
                                                   dim = ACOUSTIC_DIM,
                                                   max_len = ACOUSTIC_MAX_LEN)
                                                  for a in train_audio_data_utterance1], 0)
train_audio_data_utterance1.shape

In [ ]:
test_audio_data_utterance1 = torch.stack([pad_seq(torch.tensor(a, dtype = torch.float),
                                                   dim = ACOUSTIC_DIM,
                                                   max_len = ACOUSTIC_MAX_LEN)
                                                  for a in test_audio_data_utterance1], 0)
test_audio_data_utterance1.shape

In [ ]:
VISUAL_DIM

In [ ]:
VISUAL_MAX_LEN

In [ ]:
train_image_data_utterance1 = torch.stack([pad_seq(torch.tensor(a, dtype = torch.float),
                                                   dim = VISUAL_DIM,
                                                   max_len = VISUAL_MAX_LEN)
                                                  for a in train_image_data_utterance1], 0)
train_image_data_utterance1.shape

In [ ]:
test_image_data_utterance1 = torch.stack([pad_seq(torch.tensor(a, dtype = torch.float),
                                                   dim = VISUAL_DIM,
                                                   max_len = VISUAL_MAX_LEN)
                                                  for a in test_image_data_utterance1], 0)
test_image_data_utterance1.shape

In [ ]:
# path = "/content/drive/MyDrive/Colab Notebooks/32/datasetTrue_original/sarcasmDataset_speaker_dependent_True.npz"
# data2 = np.load(path, mmap_mode=True)

# train_audio_data_utterance1 = data2['feautesUA_train'][foldNum]
# train_image_data_utterance1 = data2['feautesUV_train'][foldNum]

# test_audio_data_utterance1 = data2['feautesUA_test'][foldNum]
# test_image_data_utterance1 = data2['feautesUV_test'][foldNum]

In [ ]:
# model =  MultiModalRobertaModel.from_pretrained('cardiffnlp/twitter-roberta-base-sentiment-latest')
# print(model)

# tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
# print(tokenizer)

# num_param = sum(p.numel() for p in model.parameters())
# print("Total parameters : ", num_param/1e6)

In [ ]:
model = MultiModalBertModel.from_pretrained('bert-base-uncased')



print(model)

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
print(tokenizer)

num_param = sum(p.numel() for p in model.parameters())
print("Total trainanable parameters : ", num_param/1e6)


In [ ]:
cnt = 0
for name, param in model.named_parameters():
    print("Count : ", cnt, " name : ", name)
    cnt+=1

In [ ]:
cnt = 0
for name, param in model.named_parameters():

    if(cnt>=197):
    # if(cnt>=389):

        param.requires_grad = True
        print("Count : ", cnt, " name : ", name)

    else:
        param.requires_grad = False
    cnt+=1


num_param = sum(p.numel() for p in model.parameters() if p.requires_grad)
print("Total trainanable parameters : ", num_param/1e6)

In [ ]:
foldNum

In [ ]:
with open("/content/drive/MyDrive/Colab Notebooks/32/json_file_fold.p", "rb") as f:
    text_file = pickle.load(f)

train_text = text_file["json_file_list_" + str(foldNum) +"_train"]
test_text =  text_file["json_file_list_" + str(foldNum) +"_test"]

In [ ]:
# Trainlen = 483

In [ ]:
print(type(train_audio_data_utterance1))
print(train_audio_data_utterance1.shape)

In [ ]:
# combined_train_data = []
# for j in range(len(train_text)):
#   temp_text = train_text[j]
#   temp_audio = train_audio_data_utterance1[j]
#   temp_image = train_image_data_utterance1[j]

#   temp_list = []
#   temp_list.append(temp_text)
#   temp_list.append(temp_audio)
#   temp_list.append(temp_image)

#   combined_train_data.append(temp_list)

# random.shuffle(combined_train_data)

In [ ]:
# train_text2 = []
# train_audio_utterance2 = []
# train_image_utterance2 = []

# for j in combined_train_data:
#   train_text2.append(j[0])
#   train_audio_utterance2.append(j[1])
#   train_image_utterance2.append(j[2])





In [ ]:
# print(len(train_text2))
# print(len(train_audio_utterance2))
# print(len(train_image_utterance2))

In [ ]:
# train_audio_data_utterance2 = torch.tensor(train_audio_utterance2)
# train_audio_data_utterance2.shape

In [ ]:
# train_image_data_utterance2 = torch.tensor(train_image_utterance2)
# train_image_data_utterance2.shape

In [ ]:
# train_audio_data_utterance = torch.tensor(train_audio_data_utterance2)[:Trainlen]
# # train_audio_data_utterance = train_audio_data_utterance.unsqueeze(dim = 1)
# train_audio_data_utterance.shape

In [ ]:
train_audio_data_utterance = torch.tensor(train_audio_data_utterance1)
# train_audio_data_utterance = train_audio_data_utterance.unsqueeze(dim = 1)
train_audio_data_utterance.shape

In [ ]:
# train_audio_data_utterance = torch.tensor(train_audio_data_utterance1)[:Trainlen]
# # train_audio_data_utterance = train_audio_data_utterance.unsqueeze(dim = 1)
# train_audio_data_utterance.shape

In [ ]:
# train_image_data_utterance = torch.tensor(train_image_data_utterance2)[:Trainlen]
# # train_image_data_utterance = train_image_data_utterance.unsqueeze(dim = 1)
# train_image_data_utterance.shape

In [ ]:
train_image_data_utterance = torch.tensor(train_image_data_utterance1)
# train_image_data_utterance = train_image_data_utterance.unsqueeze(dim = 1)
train_image_data_utterance.shape

In [ ]:
# train_image_data_utterance = torch.tensor(train_image_data_utterance1)[:Trainlen]
# # train_image_data_utterance = train_image_data_utterance.unsqueeze(dim = 1)
# train_image_data_utterance.shape

In [ ]:
# valid_audio_data_utterance = torch.tensor(train_audio_data_utterance2)[Trainlen:]
# valid_audio_data_utterance.shape

In [ ]:
len(test_text)

In [ ]:
test_image_data_utterance1.shape

In [ ]:
test_audio_data_utterance1.shape

In [ ]:
# combined_test_data = []

# for j in range(len(test_text)):
#   temp_text = test_text[j]
#   temp_audio = test_audio_data_utterance1[j]
#   temp_image = test_image_data_utterance1[j]

#   temp_list = []
#   temp_list.append(temp_text)
#   temp_list.append(temp_audio)
#   temp_list.append(temp_image)

#   combined_test_data.append(temp_list)

# random.shuffle(combined_test_data)


In [ ]:
# test_text2 = []
# test_audio_utterance2 = []
# test_image_utterance2 = []

# for j in combined_test_data:
#   test_text2.append(j[0])
#   test_audio_utterance2.append(j[1])
#   test_image_utterance2.append(j[2])


In [ ]:
# VALIDLEN = 69

In [ ]:
# len(test_audio_utterance2)

In [ ]:
# test_audio_utterance2[0].shape

In [ ]:
# torch.stack(test_audio_utterance2).shape

In [ ]:
# valid_audio_data_utterance = test_audio_data_utterance1[:VALIDLEN]
# valid_audio_data_utterance.shape

In [ ]:
# test_audio_data_utterance = test_audio_data_utterance1[VALIDLEN:]
# test_audio_data_utterance.shape

In [ ]:
# valid_image_data_utterance = torch.tensor(train_image_data_utterance2)[Trainlen:]
# valid_image_data_utterance.shape

In [ ]:
# valid_image_data_utterance = test_image_data_utterance1[:VALIDLEN]
# valid_image_data_utterance.shape

In [ ]:
# test_image_data_utterance = test_image_data_utterance1[VALIDLEN:]
# test_image_data_utterance.shape

In [ ]:
# test_audio_data_utterance = torch.tensor(test_audio_data_utterance1)
# # test_audio_data_utterance = test_audio_data_utterance.unsqueeze(dim = 1)
# test_audio_data_utterance.shape

In [ ]:
# test_image_data_utterance = torch.tensor(test_image_data_utterance1)
# # test_image_data_utterance = test_image_data_utterance.unsqueeze(dim = 1)
# test_image_data_utterance.shape

In [ ]:


# # print(len(text_file[train_text]))
# # print(train_text)
# # print(len(text_file[test_text]))

# train_audio_broadcast_utterance = audio_video_broadcast(train_audio_data_utterance)

# print("train_audio_broadcast_utterance complete : ", train_audio_broadcast_utterance.shape)
# train_image_broadcast_utterance = audio_video_broadcast(train_image_data_utterance)
# print("train_image_broadcast_utterance complete : ",train_image_broadcast_utterance.shape)

# valid_audio_broadcast_utterance = audio_video_broadcast(valid_audio_data_utterance)
# print("valid_audio_broadcast_utterance complete : ", valid_audio_broadcast_utterance.shape)

# valid_image_broadcast_utterance = audio_video_broadcast(valid_image_data_utterance)
# print('valid_image_broadcast_utterance complete : ', valid_image_broadcast_utterance.shape)

# test_audio_broadcast_utterance = audio_video_broadcast(test_audio_data_utterance)
# print("test_audio_broadcast_utterance complete : ",test_audio_broadcast_utterance.shape)
# test_image_broadcast_utterance = audio_video_broadcast(test_image_data_utterance)
# print("test_image_broadcast_utterance complete : ",test_image_broadcast_utterance.shape)

In [ ]:
# tokenizer

In [ ]:
# p = {
#         'additional_special_tokens' : ['[CONTEXT]', '[UTTERANCE]']
#     }

# tokenizer.add_special_tokens(p)

# tokenizer

In [ ]:
# model.resize_token_embeddings(len(tokenizer))

In [ ]:
p = {
        'additional_special_tokens' : ['[CONTEXT]', '[UTTERANCE]']
    }

tokenizer.add_special_tokens(p)

tokenizer

In [ ]:
# tokenizer2

In [ ]:
model.resize_token_embeddings(len(tokenizer))

In [ ]:
tokenizer

In [ ]:
# def prepare_dataset_context(text_data):


#             context = []
#             # labels = []
#             for i in range(len(text_data)):
#                 data_point = text_data[i]

#                 # example_speaker = data_point['speaker']
#                 # example_utterance = data_point['utterance']
#                 # temp_label = int(data_point['sarcasm'])

#                 # example_context = '[CONTEXT] '
#                 example_context = ''

#                 temp_len = len(data_point['context_speakers'])
#                 cnt = 0
#                 print("Temp len : ", temp_len)
#                 for speaker, utterance in list(zip(data_point['context_speakers'], data_point['context'])):
#                     print("count : ", cnt)
#                     if(cnt == temp_len - 1):
#                       example_context = example_context + speaker.upper() + " : " + utterance
#                     else:
#                       example_context = example_context + speaker.upper() + " : " + utterance + " , "
#                     cnt+=1




#                 # print(example_dialog)
#                 example_context = re.sub(' +', ' ', example_context)

#                 context.append(example_context)
#                 # labels.append(temp_label)

#             # df = pd.DataFrame(dialog, columns=['dialog'])

#             # labels = torch.tensor(labels, dtype=torch.long)



#             enc = bert_tokenizer(context, max_length = SOURCE_MAX_LEN, padding = 'max_length', truncation = True)

#             # df['audio_features'] = acoustic_data
#             # df['visual_features'] = visual_data

#             return torch.tensor(enc['input_ids'], dtype=torch.long), torch.tensor(enc['attention_mask'], dtype=torch.bool)


In [ ]:
def prepare_dataset(text_data):


            dialog = []
            labels = []
            for i in range(len(text_data)):
                data_point = text_data[i]

                example_speaker = data_point['speaker']
                example_utterance = data_point['utterance']
                temp_label = int(data_point['sarcasm'])

                # example_dialog = '[CONTEXT] '
                # example_dialog = '[TARGET] '
                example_dialog = '[CONTEXT] '


                for speaker, utterance in list(zip(data_point['context_speakers'], data_point['context'])):
                    example_dialog = example_dialog + speaker.upper() + " : " + utterance + " | "

                example_dialog = example_dialog + ' [UTTERANCE] ' + example_speaker + " : " + example_utterance + " | "
                # example_dialog = example_dialog + example_speaker + " : " + example_utterance
                # example_dialog = example_dialog + example_speaker + " : " + example_utterance
                # print(example_dialog)
                example_dialog = re.sub(' +', ' ', example_dialog)

                dialog.append(example_dialog)
                labels.append(temp_label)

            # df = pd.DataFrame(dialog, columns=['dialog'])

            labels = torch.tensor(labels, dtype=torch.long)



            # enc = tokenizer(dialog, max_length = SOURCE_MAX_LEN, padding = 'max_length', truncation = True)
            enc = tokenizer(dialog, max_length = SOURCE_MAX_LEN, padding = 'max_length', truncation = True)

            # df['audio_features'] = acoustic_data
            # df['visual_features'] = visual_data

            return torch.tensor(enc['input_ids'], dtype=torch.long), torch.tensor(enc['attention_mask'], dtype=torch.bool), labels


In [ ]:
tokenizer

In [ ]:
len(train_text)

In [ ]:
train_text_input_ids1, train_text_attention_mask1, train_ground_truth1 = prepare_dataset(train_text)
train_ground_truth1.shape

In [ ]:
# train_text_input_ids1, train_text_attention_mask1, train_ground_truth1 = prepare_dataset_utterance(train_text)
# train_ground_truth1.shape

In [ ]:
train_text_input_ids = train_text_input_ids1
train_text_input_ids.shape

In [ ]:
# train_text_input_ids = train_text_input_ids1[:Trainlen]
# train_text_input_ids.shape

In [ ]:
train_text_attention_mask = train_text_attention_mask1
train_text_attention_mask.shape

In [ ]:
# train_text_attention_mask = train_text_attention_mask1[:Trainlen]
# train_text_attention_mask.shape

In [ ]:
train_ground_truth = train_ground_truth1
train_ground_truth.shape

In [ ]:
# train_ground_truth = train_ground_truth1[:Trainlen]
# train_ground_truth.shape

In [ ]:
# train_ground_truth = train_ground_truth1[:Trainlen]
# train_ground_truth.shape

In [ ]:
# train_ground_truth

In [ ]:
# context_input_ids, context_attention_mask = prepare_dataset_context(train_text)
# print(context_input_ids.shape)

In [ ]:
# context_attention_mask.shape

In [ ]:
print("TYPE : train_text_input_ids : ", type(train_text_input_ids))

In [ ]:

# train_context_input_ids = context_input_ids
# train_context_input_ids.shape

In [ ]:

# train_context_input_ids = context_input_ids[:Trainlen]
# train_context_input_ids.shape

In [ ]:
# train_context_attention_mask = context_attention_mask
# train_context_attention_mask.shape

In [ ]:
# train_context_attention_mask = context_attention_mask[:Trainlen]
# train_context_attention_mask.shape

In [ ]:
# valid_text_input_ids = train_text_input_ids1[Trainlen:]
# valid_text_input_ids.shape

In [ ]:

# valid_text_attention_mask = train_text_attention_mask1[Trainlen:]
# valid_text_attention_mask.shape

In [ ]:
# valid_ground_truth = train_ground_truth1[Trainlen:]
# valid_ground_truth.shape

In [ ]:
# valid_text_input_ids, valid_text_attention_mask, valid_ground_truth = prepare_dataset_utterance(valid_text)
# valid_ground_truth.shape

In [ ]:
# valid_context_input_ids = context_input_ids[Trainlen:]
# valid_context_input_ids.shape

In [ ]:
# valid_context_attention_mask = context_attention_mask[Trainlen:]
# valid_context_attention_mask.shape

In [ ]:
# test_text_input_ids, test_text_attention_mask, test_ground_truth = prepare_dataset_utterance(test_text)
# test_ground_truth.shape

In [ ]:
test_text_input_ids, test_text_attention_mask, test_ground_truth = prepare_dataset(test_text)
test_ground_truth.shape

In [ ]:
# valid_id = test_text_input_ids[:VALID_LEN]
# valid_id.shape

In [ ]:
# test_id = test_text_input_ids[VALIDLEN:]
# test_id.shape

In [ ]:
# valid_mask = test_text_attention_mask[:VALIDLEN]
# valid_mask.shape

In [ ]:
# test_mask = test_text_attention_mask[VALIDLEN:]
# test_mask.shape

In [ ]:
# valid_truth = test_ground_truth[:VALIDLEN]
# valid_truth.shape

In [ ]:
# test_truth = test_ground_truth[VALIDLEN:]
# test_truth.shape

In [ ]:
# test_context_input_ids, test_context_attention_mask = prepare_dataset_context(test_text)
# test_context_input_ids.shape

In [ ]:
# valid_context_id = test_context_input_ids[:VALIDLEN]
# valid_context_id.shape

In [ ]:
# test_context_id = test_context_input_ids[VALIDLEN:]
# test_context_id.shape

In [ ]:
# test_context_attention_mask.shape

In [ ]:
# valid_context_mask = test_context_attention_mask[:VALIDLEN]
# valid_context_mask.shape

In [ ]:
# test_context_mask = test_context_attention_mask[VALIDLEN:]
# test_context_mask.shape

In [ ]:
# tokenizer.add_tokens(['[CONTEXT]', '[TARGET]'], special_tokens = True)
# model.resize_token_embeddings(len(tokenizer))

In [ ]:
test_audio_data_utterance1.shape

In [ ]:
test_image_data_utterance1.shape

In [ ]:
print(test_text_input_ids.shape)
print(test_text_attention_mask.shape)
print(test_ground_truth.shape)

In [ ]:
# print(test_context_input_ids.shape)
# print(test_context_attention_mask.shape)


In [ ]:
# test_input_data = []


# for j in range(test_ground_truth.shape[0]):
#   temp_list = []
#   temp_list.append(test_text_input_ids[j])
#   temp_list.append(test_text_attention_mask[j])
#   temp_list.append(test_context_input_ids[j])
#   temp_list.append(test_context_attention_mask[j])
#   temp_list.append(test_audio_data_utterance1[j])
#   temp_list.append(test_image_data_utterance1[j])

#   test_input_data.append(temp_list)




In [ ]:
# print(type(test_input_data))
# print(len(test_input_data))

In [ ]:
# test_output_data = test_ground_truth.tolist()
# print(type(test_output_data))
# print(len(test_output_data))

In [ ]:
# X_valid, X_test, Y_valid, Y_test = train_test_split(
#     test_input_data, test_output_data, test_size = 0.5, stratify = test_output_data
# )

In [ ]:
# len(X_valid)

In [ ]:
# len(X_test)

In [ ]:
# len(Y_valid)

In [ ]:
# len(Y_test)

In [ ]:
# valid_text_input_ids = []
# valid_text_attention_mask = []
# valid_context_input_ids = []
# valid_context_attention_mask = []

# valid_audio_data = []
# valid_image_data = []

# for j in range(len(X_valid)):
#   valid_text_input_ids.append(X_valid[j][0])
#   valid_text_attention_mask.append(X_valid[j][1])

#   valid_context_input_ids.append(X_valid[j][2])
#   valid_context_attention_mask.append(X_valid[j][3])

#   valid_audio_data.append(X_valid[j][4])

#   valid_image_data.append(X_valid[j][5])

# print(len(valid_text_input_ids))
# print(len(valid_text_attention_mask))
# print(len(valid_context_input_ids))
# print(len(valid_context_attention_mask))
# print(len(valid_audio_data))
# print(len(valid_image_data))

In [ ]:
# valid_text_input_ids[0].shape

In [ ]:
# valid_text_input_ids = torch.stack(valid_text_input_ids)
# print(valid_text_input_ids.shape)
# valid_text_attention_mask = torch.stack(valid_text_attention_mask)
# print(valid_text_attention_mask.shape)
# valid_context_input_ids = torch.stack(valid_context_input_ids)
# print(valid_context_input_ids.shape)
# valid_context_attention_mask = torch.stack(valid_context_attention_mask)
# print(valid_context_attention_mask.shape)
# valid_audio_data = torch.stack(valid_audio_data)
# print(valid_audio_data.shape)
# valid_image_data = torch.stack(valid_image_data)
# print(valid_image_data.shape)

In [ ]:
# valid_ground_truth = torch.tensor(Y_valid)
# valid_ground_truth.shape

In [ ]:
# test_text_id = []
# test_text_mask = []

# test_context_id = []
# test_context_mask = []

# test_audio_data = []
# test_image_data = []

# for j in range(len(X_test)):
#   test_text_id.append(X_test[j][0])
#   test_text_mask.append(X_test[j][1])

#   test_context_id.append(X_test[j][2])
#   test_context_mask.append(X_test[j][3])

#   test_audio_data.append(X_test[j][4])

#   test_image_data.append(X_test[j][5])

# test_text_id = torch.stack(test_text_id)
# print(test_text_id.shape)
# test_text_mask = torch.stack(test_text_mask)
# print(test_text_mask.shape)
# test_context_id = torch.stack(test_context_id)
# print(test_context_id.shape)
# test_context_mask = torch.stack(test_context_mask)
# print(test_context_mask.shape)
# test_audio_data = torch.stack(test_audio_data)
# print(test_audio_data.shape)
# test_image_data = torch.stack(test_image_data)
# print(test_image_data.shape)

In [ ]:
# test_ground_truth = torch.tensor(Y_test)
# print(test_ground_truth.shape)

In [ ]:

# tokenizer

In [ ]:
tokenizer.all_special_tokens

In [ ]:
# tokenizer.all_special_tokens


In [ ]:
print(tokenizer.all_special_ids)

In [ ]:
# train_audio_broadcast_utterance.shape

In [ ]:
# valid_audio_data_utterance = test_audio_data_utterance[:VALID_LEN, :, :]
# valid_audio_data_utterance.shape

In [ ]:
# valid_image_data_utterance = test_image_data_utterance[:VALID_LEN, :, :]
# valid_image_data_utterance.shape

In [ ]:




# test_audio_data_utterance = test_audio_data_utterance[VALID_LEN:, :, :]
# test_audio_data_utterance.shape

In [ ]:


# test_image_data_utterance = test_image_data_utterance[VALID_LEN:, :, :]
# test_image_data_utterance.shape

In [ ]:
test_input_data = []


for j in range(test_ground_truth.shape[0]):
  temp_list = []
  temp_list.append(test_text_input_ids[j])
  temp_list.append(test_text_attention_mask[j])

  temp_list.append(test_audio_data_utterance1[j])
  temp_list.append(test_image_data_utterance1[j])

  test_input_data.append(temp_list)




In [ ]:
print(type(test_input_data))
print(len(test_input_data))

In [ ]:
test_output_data = test_ground_truth.tolist()
print(type(test_output_data))
print(len(test_output_data))

In [ ]:
X_valid, X_test, Y_valid, Y_test = train_test_split(
    test_input_data, test_output_data, test_size = 0.5, stratify = test_output_data
)

In [ ]:
len(X_valid)

In [ ]:
len(X_test)

In [ ]:
len(Y_valid)

In [ ]:
len(Y_test)

In [ ]:
valid_text_input_ids = []
valid_text_attention_mask = []
valid_context_input_ids = []
valid_context_attention_mask = []

valid_audio_data = []
valid_image_data = []

for j in range(len(X_valid)):
  valid_text_input_ids.append(X_valid[j][0])
  valid_text_attention_mask.append(X_valid[j][1])


  valid_audio_data.append(X_valid[j][2])

  valid_image_data.append(X_valid[j][3])

print(len(valid_text_input_ids))
print(len(valid_text_attention_mask))

print(len(valid_audio_data))
print(len(valid_image_data))

In [ ]:
valid_text_input_ids[0].shape

In [ ]:
valid_text_input_ids = torch.stack(valid_text_input_ids)
print(valid_text_input_ids.shape)
valid_text_attention_mask = torch.stack(valid_text_attention_mask)
print(valid_text_attention_mask.shape)

valid_audio_data = torch.stack(valid_audio_data)
print(valid_audio_data.shape)
valid_image_data = torch.stack(valid_image_data)
print(valid_image_data.shape)

In [ ]:
valid_ground_truth = torch.tensor(Y_valid)
valid_ground_truth.shape

In [ ]:
test_text_id = []
test_text_mask = []

test_context_id = []
test_context_mask = []

test_audio_data = []
test_image_data = []

for j in range(len(X_test)):
  test_text_id.append(X_test[j][0])
  test_text_mask.append(X_test[j][1])


  test_audio_data.append(X_test[j][2])

  test_image_data.append(X_test[j][3])

test_text_id = torch.stack(test_text_id)
print(test_text_id.shape)
test_text_mask = torch.stack(test_text_mask)
print(test_text_mask.shape)

test_audio_data = torch.stack(test_audio_data)
print(test_audio_data.shape)
test_image_data = torch.stack(test_image_data)
print(test_image_data.shape)

In [ ]:
test_ground_truth = torch.tensor(Y_test)
print(test_ground_truth.shape)

In [ ]:
class MultimodalSarcasmDataset(Dataset):
    # def __init__(self, utterance_input_ids, utterance_attention_mask, context_input_ids, context_attention_mask, acoustic_data, visual_data, labels):
    def __init__(self, utterance_input_ids, utterance_attention_mask, acoustic_data, visual_data, labels):

        self.utterance_input_ids = utterance_input_ids
        self.utterance_attention_mask = utterance_attention_mask
        # self.context_input_ids = context_input_ids
        # self.context_attention_mask = context_attention_mask
        # self.context_attention_mask
        self.acoustic_data = acoustic_data
        self.visual_data = visual_data
        self.labels = labels

    def __len__(self):
        return len(self.utterance_input_ids)

    def __getitem__(self, idx):
        # return self.utterance_input_ids[idx], self.utterance_attention_mask[idx], self.context_input_ids[idx], self.context_attention_mask[idx], self.acoustic_data[idx], self.visual_data[idx], self.labels[idx]
        return self.utterance_input_ids[idx], self.utterance_attention_mask[idx],  self.acoustic_data[idx], self.visual_data[idx], self.labels[idx]



In [ ]:
# train_text_input_ids.shape

In [ ]:
# train_context_attention_mask.dtype

In [ ]:
# train_image_data_utterance.shape

In [ ]:
# train_text_input_ids.shape

In [ ]:
# train_text_attention_mask.shape

In [ ]:
# train_context_input_ids.shape

In [ ]:
# train_context_attention_mask.shape

In [ ]:
# train_audio_data_utterance.shape

In [ ]:
# train_image_data_utterance.shape

In [ ]:
# train_ground_truth.shape

In [ ]:
# valid_context_mask.shape

In [ ]:
# valid_context_id.shape

In [ ]:
# valid_truth.shape

In [ ]:
# test_audio_data_utterance.shape

In [ ]:
# test_context_input_ids.shape

In [ ]:
# test_audio_data_utterance1.shape


In [ ]:
test_image_data.shape

In [ ]:
test_audio_data.shape


In [ ]:
test_text_id.shape

In [ ]:
test_text_mask.shape

In [ ]:
valid_ground_truth.shape

In [ ]:
test_ground_truth.shape

In [ ]:
# train_loader = DataLoader(MultimodalSarcasmDataset(train_text_input_ids, train_text_attention_mask, train_context_input_ids, train_context_attention_mask,train_audio_broadcast_utterance, train_image_broadcast_utterance, train_ground_truth), batch_size=32, shuffle = True)
# valid_loader = DataLoader(MultimodalSarcasmDataset(valid_text_input_ids, valid_text_attention_mask, valid_context_input_ids, valid_context_attention_mask, valid_audio_broadcast_utterance, valid_image_broadcast_utterance, valid_ground_truth), batch_size = 32, shuffle = False)
# test_loader = DataLoader(MultimodalSarcasmDataset(test_text_input_ids, test_text_attention_mask, test_context_input_ids, test_context_attention_mask, test_audio_broadcast_utterance, test_image_broadcast_utterance, test_ground_truth), batch_size=32, shuffle = False)

# train_loader = DataLoader(MultimodalSarcasmDataset(train_text_input_ids, train_text_attention_mask, train_context_input_ids, train_context_attention_mask, train_audio_data_utterance, train_image_data_utterance, train_ground_truth), batch_size=32, shuffle = True)
# # valid_loader = DataLoader(MultimodalSarcasmDataset(valid_text_input_ids, valid_text_attention_mask, valid_context_input_ids, valid_context_attention_mask,   valid_audio_data, valid_image_data, valid_ground_truth), batch_size = 32, shuffle = False)
# test_loader = DataLoader(MultimodalSarcasmDataset(test_text_input_ids, test_text_attention_mask, test_context_input_ids, test_context_attention_mask,  test_audio_data_utterance1, test_image_data_utterance1, test_ground_truth), batch_size=32, shuffle = False)

# train_loader = DataLoader(MultimodalSarcasmDataset(train_text_input_ids, train_text_attention_mask,  train_audio_data_utterance, train_image_data_utterance, train_ground_truth), batch_size=32, shuffle = True)
# # valid_loader = DataLoader(MultimodalSarcasmDataset(valid_text_input_ids, valid_text_attention_mask, valid_context_input_ids, valid_context_attention_mask,   valid_audio_data, valid_image_data, valid_ground_truth), batch_size = 32, shuffle = False)
# test_loader = DataLoader(MultimodalSarcasmDataset(test_text_input_ids, test_text_attention_mask,   test_audio_data_utterance1, test_image_data_utterance1, test_ground_truth), batch_size=32, shuffle = False)

# train_loader = DataLoader(MultimodalSarcasmDataset(train_text_input_ids, train_text_attention_mask,  train_audio_data_utterance, train_image_data_utterance, train_ground_truth), batch_size=4, shuffle = True)
# # valid_loader = DataLoader(MultimodalSarcasmDataset(valid_text_input_ids, valid_text_attention_mask, valid_context_input_ids, valid_context_attention_mask,   valid_audio_data, valid_image_data, valid_ground_truth), batch_size = 32, shuffle = False)
# test_loader = DataLoader(MultimodalSarcasmDataset(test_text_input_ids, test_text_attention_mask,   test_audio_data_utterance1, test_image_data_utterance1, test_ground_truth), batch_size=4, shuffle = False)

train_loader = DataLoader(MultimodalSarcasmDataset(train_text_input_ids, train_text_attention_mask,  train_audio_data_utterance, train_image_data_utterance, train_ground_truth), batch_size=32, shuffle = True)
valid_loader = DataLoader(MultimodalSarcasmDataset(valid_text_input_ids, valid_text_attention_mask,    valid_audio_data, valid_image_data, valid_ground_truth), batch_size = 32, shuffle = False)
test_loader = DataLoader(MultimodalSarcasmDataset(test_text_id, test_text_mask,   test_audio_data, test_image_data, test_ground_truth), batch_size=32, shuffle = False)


print(test_loader)

# print(train_loader)

In [ ]:
len(train_loader.dataset)

In [ ]:
len(valid_loader.dataset)

In [ ]:
len(test_loader.dataset)

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr = LEARNING_RATE)
criterion = torch.nn.CrossEntropyLoss()

In [ ]:
model.config

In [ ]:
DEVICE

In [ ]:
model = model.to(DEVICE)
model


In [ ]:
def train_epoch(model, data_loader):
      model.train()
      epoch_train_loss = 0.0


      for step, batch in enumerate(tqdm(data_loader, desc = 'Training Iteration')):
        # for i, t in enumerate(batch):
        #     print("Inside hello")
        #     print(i, " : ", type(t))
        batch = tuple(t.to(DEVICE) for t in batch)
        # input_ids, attention_mask, context_input_ids, context_attention_mask, acoustic_input, visual_input, labels = batch
        input_ids, attention_mask,  acoustic_input, visual_input, labels = batch
        # print('\ninput ids shape : ', input_ids.shape)
        # print("attention mask shape : ", attention_mask.shape)
        # print('acoustic_input shape : ', acoustic_input.shape)
        # print('visual_input shape : ', visual_input.shape)
        optimizer.zero_grad()
        # print("Input ids shape : ", input_ids.shape)
        # print("Input ids shape : ", input_ids.shape)
        outputs = model(input_ids = input_ids,
                        attention_mask = attention_mask,

                        # context_input_ids = context_input_ids,
                        # context_attention_mask = context_attention_mask,
                        acoustic_input = acoustic_input,
                        visual_input = visual_input,
                        labels = labels)

        # outputs = model2(input_ids = input_ids,
        #                 attention_mask = attention_mask,
        #                 )
        # last_hidden_state = outputs['last_hidde,n_state']
        # print('last hidden_state shape : ', last_hidden_state.shape)
        loss = outputs['loss']
        epoch_train_loss += loss.item()

        # print("Batch wise loss : ", epoch_train_loss)

        loss.backward()
        optimizer.step()



      print("Epoch train loss : ", epoch_train_loss)

In [ ]:
def valid_epoch(model, data_loader):
  model.eval()
  predictions = []
  gold = []

  valid_loss = 0.0
  with torch.no_grad():
    for step, batch in enumerate(tqdm(data_loader)):
      batch = tuple(t.to(DEVICE) for t in batch)
      # input_ids, attention_mask, context_input_ids, context_attention_mask, acoustic_input, visual_input, labels = batch
      input_ids, attention_mask,  acoustic_input, visual_input, labels = batch


      outputs = model(input_ids = input_ids,
                            attention_mask = attention_mask,
                            # context_input_ids = context_input_ids,
                            # context_attention_mask = context_attention_mask,
                            acoustic_input = acoustic_input,
                            visual_input = visual_input,
                            labels = labels)

      logits = outputs['logits']
      loss = outputs['loss']

      valid_loss += loss.item()



      pred = logits.argmax(dim = -1)

      predictions.extend(pred.tolist())
      gold.extend(labels.tolist())

  return valid_loss, predictions, gold



In [ ]:


def test_epoch(model, data_loader):
    model.eval()
    predictions = []
    gold = []

    correct = 0
    with torch.no_grad():
        for step, batch in enumerate(tqdm(data_loader)):
            batch = tuple(t.to(DEVICE) for t in batch)
            # input_ids, attention_mask, context_input_ids, context_attention_mask, acoustic_input, visual_input, labels = batch
            input_ids, attention_mask,  acoustic_input, visual_input, labels = batch

            # print("attention mask shape : ", attention_mask.shape)

            outputs = model(input_ids = input_ids,
                            attention_mask = attention_mask,
                            # context_input_ids = context_input_ids,
                            # context_attention_mask = context_attention_mask,

                            acoustic_input = acoustic_input,
                            visual_input = visual_input,
                            labels = labels)

            logits = outputs['logits']

            pred = logits.argmax(dim = -1)

            predictions.extend(pred.tolist())

            gold.extend(labels.tolist())

            correct += int((pred == labels).sum())

    return correct/len(data_loader.dataset), predictions, gold

In [ ]:
class EarlyStopping:
  def __init__(self, patience, min_delta):
    self.patience = patience
    self.min_delta = min_delta
    self.counter = 0
    self.min_validation = np.inf

  def early_stop(self, valid_loss):
    if valid_loss < self.min_validation:
      self.min_validation = valid_loss
      self.counter = 0
    elif valid_loss > (self.min_validation + self.min_delta):
      self.counter += 1
      if self.counter >= self.patience:
        return True
    return False

In [ ]:
early_stopper = EarlyStopping(patience = 15, min_delta = 0.2)
early_stopper

In [ ]:

def train_and_validation(model, train_loader, valid_loader):
  # lowest_loss = 1e6
  best_f1 = 0.0
  for epoch in range(30):
    print("\n=============Epoch : ", epoch)
    train_epoch(model, train_loader)
    valid_loss, valid_pred, valid_gold = valid_epoch(model, valid_loader)

    if early_stopper.early_stop(valid_loss):
      break

    print("Length of predictions : ", len(valid_pred))
    print("Length of gold : ", len(valid_gold))
    print("Valid loss : ", valid_loss)
    print("\n Valid Accuracy : ", accuracy_score(valid_gold, valid_pred))
    print("\n Valid Precision : ", precision_score(valid_gold, valid_pred, average = 'weighted'))
    print("\n Valid Recall : ", recall_score(valid_gold, valid_pred, average = 'weighted'))
    print("\nValid F1 score : ", f1_score(valid_gold, valid_pred, average = 'weighted'))


    curr_f1 = f1_score(valid_gold, valid_pred, average = 'weighted')

    curr_loss = valid_loss
    # if((curr_f1 > best_f1) and (epoch>=4)):
    if(curr_f1 > best_f1):
    # if(curr_loss < lowest_loss):
      best_f1 = curr_f1
      # print("Valid pred : ", valid_pred)
      # print('valid_gold : ', valid_gold)
      # torch.save(model.state_dict(), '/content/drive/MyDrive/Colab Notebooks/32/saved_model_f1/roberta/new_seed/best_model_epoch_'+str(epoch)+'_best_f1_'+str(int(best_f1*100))+'_foldNum_'+str(foldNum)+'.pt')
      torch.save(model.state_dict(), '/content/drive/MyDrive/Colab Notebooks/32/saved_model/bert_only/best_model_epoch_'+str(epoch)+'_best_f1_'+str(int(best_f1*100))+'_foldNum_'+str(foldNum)+'.pt')

      print("model saved\n")
      # print("best model\n")




In [ ]:
# train_and_validation(model, train_loader, test_loader)
train_and_validation(model, train_loader, valid_loader)

In [ ]:
# best_model_epoch_12_best_f1_72_foldNum_0.pt
# best_model_epoch_9_best_f1_69_foldNum_1.pt
# best_model_epoch_9_best_f1_79_foldNum_2.pt
# best_model_epoch_9_best_f1_73_foldNum_3.pt
# best_model_epoch_7_best_f1_75_foldNum_4.pt

In [ ]:
path = '/content/drive/MyDrive/Colab Notebooks/32/saved_model/bert_only/best_model_epoch_19_best_f1_81_foldNum_0.pt'
#

In [ ]:
# PATH_0 = '/content/drive/MyDrive/Colab Notebooks/32/saved_model_f1/best_model_epoch_12_f1_76.pt'
# PATH_1 = '/content/drive/MyDrive/Colab Notebooks/32/saved_model_f1/best_model_epoch_13_f1_78_foldNum_1.pt'
# PATH_2 = '/content/drive/MyDrive/Colab Notebooks/32/saved_model_f1/best_model_epoch_9_f1_64_foldNum_2.pt'
# PATH_3 = '/content/drive/MyDrive/Colab Notebooks/32/saved_model_f1/best_model_epoch_5_f1_73_foldNum_3.pt'
# PATH_4 = '/content/drive/MyDrive/Colab Notebooks/32/saved_model_f1/best_model_epoch_4_f1_74_foldNum_4.pt'

In [ ]:

# PATH_0 = '/content/drive/MyDrive/Colab Notebooks/32/saved_model_f1/best_model_epoch_6_f1_50_foldNum_0.pt'
# PATH_1 = '/content/drive/MyDrive/Colab Notebooks/32/saved_model_f1/best_model_epoch_4_f1_70_foldNum_1.pt'
# PATH_2 = '/content/drive/MyDrive/Colab Notebooks/32/saved_model_f1/best_model_epoch_11_f1_63_foldNum_2.pt'
# PATH_3 = '/content/drive/MyDrive/Colab Notebooks/32/saved_model_f1/best_model_epoch_4_f1_44_foldNum_3.pt'
# PATH_4 = '/content/drive/MyDrive/Colab Notebooks/32/saved_model_f1/best_model_epoch_4_f1_62_foldNum_4.pt'

In [ ]:
foldNum

In [ ]:
model.load_state_dict(torch.load(path))

In [ ]:
# 2*3

In [ ]:


acc, test_pred, test_gold = test_epoch(model, test_loader)

print(acc)

print("\nAccuracy : ", accuracy_score(test_gold, test_pred))
print("\nPrecision : ", precision_score(test_gold, test_pred, average = 'weighted'))
print("\nRecall : ", recall_score(test_gold, test_pred, average = 'weighted'))
print("\nF1 score : ", f1_score(test_gold, test_pred, average = 'weighted'))















In [ ]:
# valid_loss, valid_pred, valid_gold = valid_epoch(model, valid_loader)

# # print(acc)

# print("\nAccuracy : ", accuracy_score(valid_gold, valid_pred))
# print("\nPrecision : ", precision_score(valid_gold, valid_pred, average = 'weighted'))
# print("\nRecall : ", recall_score(valid_gold, valid_pred, average = 'weighted'))
# print("\nF1 score : ", f1_score(valid_gold, valid_pred, average = 'weighted'))

In [ ]:
# valid_loss, valid_pred, valid_gold = valid_epoch(model, valid_loader)

# # print(acc)

# print("\nAccuracy : ", accuracy_score(valid_gold, valid_pred))
# print("\nPrecision : ", precision_score(valid_gold, valid_pred))
# print("\nRecall : ", recall_score(valid_gold, valid_pred))
# print("\nF1 score : ", f1_score(valid_gold, valid_pred))

In [ ]:
# valid_loss, valid_pred, valid_gold = valid_epoch(model, valid_loader)

# # print(acc)

# print("\nAccuracy : ", accuracy_score(valid_gold, valid_pred))
# print("\nPrecision : ", precision_score(valid_gold, valid_pred, average = 'micro'))
# print("\nRecall : ", recall_score(valid_gold, valid_pred, average = 'micro'))
# print("\nF1 score : ", f1_score(valid_gold, valid_pred, average = 'micro'))

In [ ]:


# acc, test_pred, test_gold = test_epoch(model, test_loader)

# # print(acc)

# print("\nAccuracy : ", accuracy_score(test_gold, test_pred))
# print("\nPrecision : ", precision_score(test_gold, test_pred))
# print("\nRecall : ", recall_score(test_gold, test_pred))
# print("\nF1 score : ", f1_score(test_gold, test_pred))















In [ ]:


# acc, test_pred, test_gold = test_epoch(model, test_loader)

# # print(acc)

# print("\nAccuracy : ", accuracy_score(test_gold, test_pred))
# print("\nPrecision : ", precision_score(test_gold, test_pred, average = 'micro'))
# print("\nRecall : ", recall_score(test_gold, test_pred, average = 'micro'))
# print("\nF1 score : ", f1_score(test_gold, test_pred, average = 'micro'))















In [ ]:
# test_pred

In [ ]:
# test_gold